In [1]:
import pandas as pd
import numpy as np
df_category = pd.read_csv('exports/category_skill.csv')
df_job = pd.read_csv('exports/job_skill.csv')

In [2]:
import requests
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)


# Import prepared data directly to database

---

## Connect to MongoDB

In [3]:
from pymongo import MongoClient

# build a new client instance of MongoClient
mongo_client = MongoClient('localhost', 27017)

# connect database
db = mongo_client.skillguider

In [4]:
# drop collections before import new data
db['skills'].drop()
db['categories'].drop()
db['jobs'].drop()

---

## Import all skills to database

### Preparing data

In [5]:
df_all_skill = pd.concat([df_category, df_job], axis=0)

df_all_skill = df_all_skill.drop(['sum', 'count', 'priority', 'job', 'category'], axis=1).drop_duplicates().reset_index(drop=True)
df_all_skill = df_all_skill.rename(columns={'skill': 'title'})
df_all_skill.loc[df_all_skill.index[:], 'title'] = df_all_skill['title'].str.capitalize()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_all_skill)

,title
0,Python
1,Sql
2,Java
3,Computer science
4,Machine learning
5,Analytics
6,Javascript
7,Mysql
8,Selenium
9,Photoshop


In [6]:
df_keyword_old = pd.read_csv('exports/keyword.csv')
df_keyword_new = pd.merge(df_keyword_old, df_all_skill, on='title', how='right').replace(np.nan, '', regex=True)
df_keyword_new.to_csv('exports/keyword.csv', index=False)

In [7]:
skill_json = df_keyword_new.to_json('exports/temp/all_skills.json', orient='records', default_handler=str)

In [8]:
with open('exports/temp/all_skills.json') as f:
    data = json.load(f)

db['skills'].insert_many(data)

---

## Import categories to database

### Preparing data

In [9]:
cursor = db['skills'].find({})
df_skill_with_id = pd.DataFrame(list(cursor), columns = ['_id', 'title'])

In [10]:
df_skill_with_id = df_skill_with_id.rename(columns={'title': 'skill'})
df_skill_with_id.loc[df_skill_with_id.index[:], 'skill'] = df_skill_with_id['skill'].str.lower()
df_skill_with_id

,_id,skill
0,606a8e154abc466c9d5dc3b9,python
1,606a8e154abc466c9d5dc3ba,sql
2,606a8e154abc466c9d5dc3bb,java
3,606a8e154abc466c9d5dc3bc,computer science
4,606a8e154abc466c9d5dc3bd,machine learning
...,...,...
116,606a8e154abc466c9d5dc42d,consulting
117,606a8e154abc466c9d5dc42e,debugging
118,606a8e154abc466c9d5dc42f,gprs
119,606a8e154abc466c9d5dc430,gsm


In [11]:
df_category_skill = df_category.drop(['sum', 'count'], axis=1).reset_index(drop=True)

In [12]:
df_merge_category = pd.merge(df_category_skill, df_skill_with_id, on='skill').sort_values(by=['category', 'skill'], ascending=False).reset_index(drop=True)
df_merge_category = df_merge_category.drop(['skill'], axis=1).rename(columns={'_id': 'skill_id', 'category': 'title'})
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_merge_category)

,title,skill_id
0,Tester,606a8e154abc466c9d5dc3ba
1,Tester,606a8e154abc466c9d5dc3c1
2,Tester,606a8e154abc466c9d5dc3b9
3,Tester,606a8e154abc466c9d5dc3bf
4,Tester,606a8e154abc466c9d5dc3bb
5,Tester,606a8e154abc466c9d5dc3bc
6,Support,606a8e154abc466c9d5dc3c9
7,Support,606a8e154abc466c9d5dc3ba
8,Support,606a8e154abc466c9d5dc3bf
9,Support,606a8e154abc466c9d5dc3c4


### Convert to json file on collection format

In [13]:
columns = df_merge_category.columns.difference(['title'])
category_json = df_merge_category.groupby(['title'])[columns].apply(lambda x: x.to_dict('r')).reset_index(name='skillset').to_json('exports/temp/categories.json', orient='records', default_handler=str)

/usr/local/anaconda3/envs/ml101/lib/python3.9/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


### Insert json data to MongoDB

In [14]:
with open('exports/temp/categories.json') as f:
    data = json.load(f)

db['categories'].insert_many(data)

---

## Import jobs to database

### Preparing data

In [15]:
cursor = db['categories'].find({})
df_category_with_id = pd.DataFrame(list(cursor), columns = ['_id', 'title'])
df_category_with_id = df_category_with_id.rename(columns={'title': 'category'})

In [16]:
df_job_skill = df_job.drop(['sum'], axis=1).reset_index(drop=True)

In [17]:
df_merge_job = pd.merge(df_job_skill, df_skill_with_id, on='skill')
df_merge_job = df_merge_job.rename(columns={'_id': 'skill_id'})

In [18]:
df_merge_job = pd.merge(df_merge_job, df_category_with_id, on='category').sort_values(by=['job', 'skill'], ascending=False).reset_index(drop=True)
df_merge_job = df_merge_job.rename(columns={'_id': 'category_id'})

In [19]:
df_merge_job = df_merge_job.drop(['skill', 'category'], axis=1).rename(columns={'job': 'title'})

In [20]:
df_merge_job['description'] = '' 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_merge_job.head())

,priority,title,skill_id,category_id,description
0,Normal,iOS,606a8e154abc466c9d5dc402,606a8e1f4abc466c9d5dc435,
1,Normal,iOS,606a8e154abc466c9d5dc404,606a8e1f4abc466c9d5dc435,
2,High,iOS,606a8e154abc466c9d5dc3fe,606a8e1f4abc466c9d5dc435,
3,Normal,iOS,606a8e154abc466c9d5dc405,606a8e1f4abc466c9d5dc435,
4,High,iOS,606a8e154abc466c9d5dc3ff,606a8e1f4abc466c9d5dc435,


### Convert to json file on collection format

In [21]:
columns = df_merge_job.columns.difference(['title', 'category_id', 'description'])
job_json = df_merge_job.groupby(['title', 'category_id', 'description'])[columns].apply(lambda x: x.to_dict('r')).reset_index(name='skillset').to_json('exports/temp/jobs.json', orient='records', default_handler=str)

/usr/local/anaconda3/envs/ml101/lib/python3.9/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


### Insert json data to MongoDB

In [22]:
with open('exports/temp/jobs.json') as f:
    data = json.load(f)

db['jobs'].insert_many(data)

In [23]:
mongo_client.close()